# Baseline

In [1]:
import pandas as pd

from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## Загрузка данных

In [2]:
train_df = pd.read_parquet('data/train.parquet')
test_df = pd.read_parquet('data/test.parquet')

In [3]:
train_df.head()

,id,rko_start_months,max_end_fact_fin_deals,max_end_plan_non_fin_deals,max_start_fin_deals,max_start_non_fin_deals,min_end_fact_fin_deals,min_end_plan_non_fin_deals,min_start_fin_deals,min_start_non_fin_deals,...,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,target_1,target_2,total_target
0,0,48.871217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.064993,33196.020871,1.229488,0.396969,323523.957625,12.420855,9.069157,1,1,1
1,1,19.530576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.270542,33195.488147,1.229488,-0.137339,145641.572203,2.124925,2.069157,0,0,0
2,2,9.706201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.431839,265071.236918,4.229488,3.595162,125727.062161,0.999583,1.069157,0,0,0
3,3,92.569902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.990581,495094.276542,9.229488,8.582252,370392.322955,3.066714,3.069157,0,0,0
4,4,26.538856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.393927,33196.317418,1.229488,0.276606,125726.775788,1.219935,1.069157,0,0,0


In [4]:
test_df.head()

,id,rko_start_months,max_end_fact_fin_deals,max_end_plan_non_fin_deals,max_start_fin_deals,max_start_non_fin_deals,min_end_fact_fin_deals,min_end_plan_non_fin_deals,min_start_fin_deals,min_start_non_fin_deals,...,cnt_days_deb_g_oper_3m,sum_cred_g_oper_3m,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m
0,360000,61.061808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.531871,51005.755087,4.275226,4.257042,33195.968822,1.229488,0.359263,301442.064297,7.049883,7.069157
1,360001,16.803743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.531871,11.888083,0.275226,0.257042,33195.968822,1.229488,0.359263,125727.663401,1.049883,1.069157
2,360002,15.448904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.531871,11.888083,0.275226,0.257042,33195.968822,1.229488,0.359263,125727.663401,1.049883,1.069157
3,360003,11.287614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.531871,11.888083,0.275226,0.257042,33195.968822,1.229488,0.359263,125727.663401,1.049883,1.069157
4,360004,7.997291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.531871,11.888083,0.275226,0.257042,135758.248681,2.229488,1.359263,126742.064297,2.049883,2.069157


## Обработка данных

Для базовой модели отбросим отдельные таргеты и будем использовать только total_target.

In [5]:
train_df.drop(["id", "target_1", "target_2"], axis=1, inplace=True)

Преобразуем тип категориальных признаков

In [6]:
cat_cols = [
    'channel_code', 'city', 'city_type',
    'index_city_code', 'ogrn_month', 'ogrn_year',
    'branch_code', 'okved', 'segment'
]

In [7]:
train_df[cat_cols] = train_df[cat_cols].astype("category")

## Разбиение на train, validation

In [85]:
X_train, X_test,y_train, y_test = train_test_split(train_df.drop('total_target', axis = 1), train_df.total_target, random_state=30, test_size= 0.2)

In [8]:
# tuned_params = {**study.best_params}
tuned_params = {'lambda_l1': 0.5349409254631267,
 'lambda_l2': 0.49856117774439324,
 'num_leaves': 512,
 'feature_fraction': 0.42939817341414027,
 'bagging_fraction': 0.5006754316465933,
 'min_child_samples': 30,
 'learning_rate': 0.03166155472455522,
 'n_estimators': 410}


In [9]:
test_df[cat_cols] = test_df[cat_cols].astype("category")

In [10]:
# from sklearn.model_selection import StratifiedGroupKFold
# from sklearn.metrics import roc_auc_score, classification_report
# import xgboost as xgb
# import numpy as np
# def my_cross_valid(X, y):
#     kf = StratifiedGroupKFold(n_splits=10)
#     models = []
#     metrics = []
#     X = np.array(X)
#     y = np.array(y)
#     counter = 0
#     for train_index, test_index in kf.split(X, y):
#         print(counter)
#         counter += 1
  
#         X_train, X_test = X[train_index], X[test_index]
#         y_train, y_test = y[train_index], y[test_index]
#         # positive_examples = np.sum(y_train)
#         # negative_examples = len(y_train) - np.sum(y_train)
#         # scale_pos_weight = (negative_examples)/(positive_examples)
#         model = LGBMClassifier(**tuned_params)
#         model.fit(X_train, y_train)

#         # print('score:', roc_auc_score(y_test, predicts))
#         models.append(model)
#         metrics.append(metrics)
#     return models, metrics
# models, metrics = my_cross_valid(train_df.drop(['total_target'], axis = 1), train_df.total_target)
# predicts = []
# probas = np.zeros(len(X_test))
# for model in models:
#     print(np.array(model.predict_proba(X_test)).shape)
#     probas += model.predict_proba(X_test)[:,1] * 0.1
    
#     predicts.append(model.predict(X_test))



# predicts = np.round(np.sum(np.array(predicts).T, axis = 1)/10)
# print(classification_report(y_test, predicts), roc_auc_score(y_test, probas))
# probas
    


## Обучение базовой модели

В качестве базовой модели возьмем LGBMClassifier

In [11]:

mean_opp_c_1m = train_df[train_df['total_target'] == 1]['sum_c_oper_1m'].mean()

train_df['Ind_sum_c_oper_1m'] = train_df['sum_c_oper_1m'].apply(lambda x: 1 if x < mean_opp_c_1m else 0)
test_df['Ind_sum_c_oper_1m'] = test_df['sum_c_oper_1m'].apply(lambda x: 1 if x < mean_opp_c_1m else 0)

# model = LGBMClassifier(num_leaves=num_leaves)
# model.fit(X_train.drop('total_target', axis = 1), y_train)
# roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

In [12]:

mean_deb_sum_g_1m = train_df[train_df['total_target'] == 1]['sum_deb_g_oper_1m'].mean()

train_df['Ind_sum_g_oper_1m'] = train_df['sum_deb_g_oper_1m'].apply(lambda x: 1 if x < mean_deb_sum_g_1m else 0)
test_df['Ind_sum_g_oper_1m'] = test_df['sum_deb_g_oper_1m'].apply(lambda x: 1 if x < mean_deb_sum_g_1m else 0)

# model = LGBMClassifier(num_leaves=num_leaves)
# model.fit(X_train.drop(['total_target'], axis = 1), y_train)
# roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

In [13]:

mean_deb_sum_g_1m = train_df[train_df['total_target'] == 1]['sum_deb_f_oper_1m'].mean()

train_df['Ind_sum_f_oper_1m'] = train_df['sum_deb_f_oper_1m'].apply(lambda x: 1 if x < mean_deb_sum_g_1m else 0)
test_df['Ind_sum_f_oper_1m'] = test_df['sum_deb_f_oper_1m'].apply(lambda x: 1 if x < mean_deb_sum_g_1m else 0)

# model = LGBMClassifier(num_leaves = num_leaves)
# model.fit(X_train.drop(['total_target'], axis = 1), y_train)
# roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

In [14]:

sum_deb_sum_e_3m = train_df[train_df['total_target'] == 1]['sum_deb_e_oper_3m'].mean()
train_df['Ind_sum_e_oper_3m'] = train_df['sum_deb_e_oper_3m'].apply(lambda x: 1 if x < sum_deb_sum_e_3m else 0)
test_df['Ind_sum_e_oper_3m'] = test_df['sum_deb_e_oper_3m'].apply(lambda x: 1 if x < sum_deb_sum_e_3m else 0)

# model = LGBMClassifier(num_leaves = num_leaves)
# model.fit(X_train.drop(['total_target'], axis = 1), y_train)
# roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

In [15]:

sum_deb_sum_g_3m = train_df[train_df['total_target'] == 1]['sum_deb_g_oper_3m'].mean()

train_df['Ind_sum_g_oper_3m'] = train_df['sum_deb_g_oper_3m'].apply(lambda x: 1 if x < sum_deb_sum_g_3m else 0)
test_df['Ind_sum_g_oper_3m'] = test_df['sum_deb_g_oper_3m'].apply(lambda x: 1 if x < sum_deb_sum_g_3m else 0)

# model = LGBMClassifier(num_leaves = num_leaves)
# model.fit(X_train.drop(['total_target'], axis = 1), y_train)
# roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

In [16]:

cnt_c_oper_3m = train_df[train_df['total_target'] == 1]['cnt_c_oper_3m'].mean()

train_df['Ind_cnt_c_oper_3m'] = train_df['cnt_c_oper_3m'].apply(lambda x: 1 if x < cnt_c_oper_3m else 0)
test_df['Ind_cnt_c_oper_3m'] = test_df['cnt_c_oper_3m'].apply(lambda x: 1 if x < cnt_c_oper_3m else 0)

# model = LGBMClassifier(num_leaves = num_leaves)
# model.fit(X_train.drop(['total_target'], axis = 1), y_train)
# roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

In [17]:

cnt_deb_d_oper_1m = train_df[train_df['total_target'] == 1]['cnt_deb_d_oper_1m'].mean()

train_df['Ind_cnt_d_oper_1m'] = train_df['cnt_deb_d_oper_3m'].apply(lambda x: 1 if x < cnt_deb_d_oper_1m else 0)
test_df['Ind_cnt_d_oper_1m'] = test_df['cnt_deb_d_oper_3m'].apply(lambda x: 1 if x < cnt_deb_d_oper_1m  else 0)

# model = LGBMClassifier(num_leaves = num_leaves)
# model.fit(X_train.drop(['total_target'], axis = 1), y_train)
# roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

In [ ]:
train_df['max_end_fact_fin_deals_isna'] = train_df['max_end_fact_fin_deals'].isna()
test_df['max_end_fact_fin_deals_isna'] = test_df['max_end_fact_fin_deals'].isna()

train_df['max_end_plan_non_fin_deals_isna'] = train_df['max_end_plan_non_fin_deals'].isna()
test_df['max_end_plan_non_fin_deals_isna'] = test_df['max_end_plan_non_fin_deals'].isna()

train_df['max_start_fin_deals_isna'] = train_df['max_start_fin_deals'].isna()
test_df['max_start_fin_deals_isna'] = test_df['max_start_fin_deals'].isna()

train_df['max_start_non_fin_deals_isna'] = train_df['max_start_non_fin_deals'].isna()
test_df['max_start_non_fin_deals_isna'] = test_df['max_start_non_fin_deals'].isna()

train_df['min_end_fact_fin_deals_isna'] = train_df['min_end_fact_fin_deals'].isna()
test_df['min_end_fact_fin_deals_isna'] = test_df['min_end_fact_fin_deals'].isna()

train_df['min_end_plan_non_fin_deals_isna'] = train_df['min_end_plan_non_fin_deals'].isna()
test_df['min_end_plan_non_fin_deals_isna'] = test_df['min_end_plan_non_fin_deals'].isna()

train_df['min_start_fin_deals_isna'] = train_df['min_start_fin_deals'].isna()
test_df['min_start_fin_deals_isna'] = test_df['min_start_fin_deals'].isna()

train_df['min_start_non_fin_deals_isna'] = train_df['min_start_non_fin_deals'].isna()
test_df['min_start_non_fin_deals_isna'] = test_df['min_start_non_fin_deals'].isna()

In [18]:
import lightgbm as lgb
def LightGBM(params, X_train_adv, X_valid_adv, y_train_adv, y_valid_adv):
    # Set data
    lgb_train = lgb.Dataset(X_train_adv, y_train_adv)
    lgb_valid = lgb.Dataset(X_valid_adv, y_valid_adv, reference = lgb_train)
    # Training
    model = lgb.train(
        params,
        lgb_train,
        valid_sets = [lgb_train, lgb_valid],
        num_boost_round = 2000,


    )
    # Prediction
    y_pred = model.predict(X_valid_adv, num_iteration = model.best_iteration)
    # Evaluation
    ROC_AUC_Score = roc_auc_score(y_valid_adv,y_pred)
    print('ROC AUC Score of LightGBM =', ROC_AUC_Score)
    return ROC_AUC_Score

In [23]:
def objective(trial):
    params = {
        'task': 'train',
        'objective': 'binary',
        'verbose': -1,
        'boosting_type': 'gbdt',
        'seed': 42,
        'metric': 'AUC',
        'is_unbalance': True,
        # 'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'num_threads': 4,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.1, 0.9),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.1, 0.9),
        'num_leaves': trial.suggest_int('num_leaves', 2, 1000),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4,
                                                  0.8),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4,
                                                  0.8),
        'min_child_samples': trial.suggest_int('min_child_samples', 25, 50),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000)
    }

    return LightGBM(params, X_train, X_test, y_train, y_test)

In [24]:
X_train, X_test,y_train, y_test = train_test_split(train_df.drop('total_target', axis = 1), train_df.total_target, random_state=30, test_size= 0.2)

In [25]:
import optuna

from optuna.samplers import TPESampler

study = optuna.create_study(direction = 'maximize', sampler = TPESampler(seed=42))
study.optimize(objective, n_trials = 40)

[I 2023-11-30 20:50:28,486] A new study created in memory with name: no-name-e4e16f19-f050-4961-a2da-3b9e1d56f666
/tmp/ipykernel_75408/1226370266.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.1, 0.9),
/tmp/ipykernel_75408/1226370266.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.1, 0.9),
/tmp/ipykernel_75408/1226370266.py:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform(

ROC AUC Score of LightGBM = 0.8982300774318328


/tmp/ipykernel_75408/1226370266.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.1, 0.9),
/tmp/ipykernel_75408/1226370266.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.1, 0.9),
/tmp/ipykernel_75408/1226370266.py:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4,
/tmp/ipykernel_75408/1226370266.py:19: FutureWarning: suggest_uniform has been deprecated

ROC AUC Score of LightGBM = 0.8687531204088663


/tmp/ipykernel_75408/1226370266.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.1, 0.9),
/tmp/ipykernel_75408/1226370266.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.1, 0.9),
/tmp/ipykernel_75408/1226370266.py:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4,
/tmp/ipykernel_75408/1226370266.py:19: FutureWarning: suggest_uniform has been deprecated

...

ROC AUC Score of LightGBM = 0.9205489860168916


/tmp/ipykernel_75408/1226370266.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.1, 0.9),
/tmp/ipykernel_75408/1226370266.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.1, 0.9),
/tmp/ipykernel_75408/1226370266.py:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4,
/tmp/ipykernel_75408/1226370266.py:19: FutureWarning: suggest_uniform has been deprecated

ROC AUC Score of LightGBM = 0.9166095996831447


/tmp/ipykernel_75408/1226370266.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.1, 0.9),
/tmp/ipykernel_75408/1226370266.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.1, 0.9),
/tmp/ipykernel_75408/1226370266.py:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4,
/tmp/ipykernel_75408/1226370266.py:19: FutureWarning: suggest_uniform has been deprecated

ROC AUC Score of LightGBM = 0.9154305933406144


In [26]:
tuned_params = {**study.best_params}
tuned_params

{'lambda_l1': 0.7296456333862358,
 'lambda_l2': 0.7650090793461436,
 'num_leaves': 882,
 'feature_fraction': 0.43644908397209087,
 'bagging_fraction': 0.4755353435998187,
 'min_child_samples': 25,
 'learning_rate': 0.014569923732383923,
 'n_estimators': 928}

In [27]:
from sklearn.model_selection import StratifiedGroupKFold, KFold
from sklearn.metrics import roc_auc_score, classification_report
import xgboost as xgb
import numpy as np
def my_cross_valid(X, y):
    kf = KFold(n_splits=10)
    models = []
    metrics = []
    X = np.array(X)
    y = np.array(y)
    counter = 0
    for train_index, test_index in kf.split(X, y):
        print(counter)
        counter += 1
  
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # positive_examples = np.sum(y_train)
        # negative_examples = len(y_train) - np.sum(y_train)
        # scale_pos_weight = (negative_examples)/(positive_examples)
        model = LGBMClassifier(**tuned_params)
        model.fit(X_train, y_train)

        # print('score:', roc_auc_score(y_test, predicts))
        models.append(model)
        metrics.append(metrics)
    return models, metrics
models, metrics = my_cross_valid(train_df.drop(['total_target'], axis = 1), train_df.total_target)
# predicts = []
# probas = np.zeros(len(X_test))
# for model in models:
#     print(np.array(model.predict_proba(X_test)).shape)
#     probas += model.predict_proba(X_test)[:,1] * 0.1
    
#     predicts.append(model.predict(X_test))



# predicts = np.round(np.sum(np.array(predicts).T, axis = 1)/10)
# print(classification_report(y_test, predicts), roc_auc_score(y_test, probas))
# probas

0
[LightGBM] [Warning] feature_fraction is set=0.43644908397209087, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.43644908397209087
[LightGBM] [Warning] lambda_l2 is set=0.7650090793461436, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7650090793461436
[LightGBM] [Warning] lambda_l1 is set=0.7296456333862358, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7296456333862358
[LightGBM] [Warning] bagging_fraction is set=0.4755353435998187, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4755353435998187
[LightGBM] [Warning] feature_fraction is set=0.43644908397209087, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.43644908397209087
[LightGBM] [Warning] lambda_l2 is set=0.7650090793461436, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7650090793461436
[LightGBM] [Warning] lambda_l1 is set=0.7296456333862358, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7296456333862358
[Ligh

In [28]:
# X_test

Качество получилось довольно неплохим, но его еще можно улучшить

## Выгрузка результатов

In [29]:
# import catboost
# model = catboost.CatBoostClassifier()
# train_df[cat_cols] = train_df[cat_cols].astype(str)
# test_df[cat_cols] = test_df[cat_cols].astype(str)
# model.fit(train_df.drop('total_target', axis = 1), train_df.total_target)


In [30]:
# X_test[cat_cols] = X_test[cat_cols].astype(str)
# roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

In [31]:

# model = LGBMClassifier()
# model.fit(X_train, y_train)
# roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

In [32]:
predicts = []
probas = np.zeros(len(test_df))
for model in models:
    # print(np.array(model.predict_proba(test_df)).shape)
    probas += model.predict_proba(test_df.drop('id', axis = 1))[:,1] * 0.1
    
    predicts.append(model.predict(test_df.drop('id', axis =1 )))

[LightGBM] [Warning] feature_fraction is set=0.43644908397209087, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.43644908397209087
[LightGBM] [Warning] lambda_l2 is set=0.7650090793461436, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7650090793461436
[LightGBM] [Warning] lambda_l1 is set=0.7296456333862358, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7296456333862358
[LightGBM] [Warning] bagging_fraction is set=0.4755353435998187, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4755353435998187
[LightGBM] [Warning] feature_fraction is set=0.43644908397209087, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.43644908397209087
[LightGBM] [Warning] lambda_l2 is set=0.7650090793461436, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7650090793461436
[LightGBM] [Warning] lambda_l1 is set=0.7296456333862358, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.7296456333862358
[LightG

In [33]:
# test_df['channel_code'] = test_df['channel_code'].apply(lambda x: 0 if x == 'НО ВКМ' or x == 'НОКМП0-350'  else x)

In [34]:
test_score = probas

In [35]:
sample_submission_df = pd.read_csv("sample_submission.csv")

In [36]:
sample_submission_df.head()

,id,score
0,360000,0.5
1,360001,0.5
2,360002,0.5
3,360003,0.5
4,360004,0.5


In [37]:
sample_submission_df["score"] = test_score

In [38]:
sample_submission_df.head()

,id,score
0,360000,0.008664
1,360001,0.010472
2,360002,0.032259
3,360003,0.025132
4,360004,0.015269


In [39]:
sample_submission_df.to_csv("my_submission.csv", index=False)